# 🌞**Évaluation et Prévision des Énergies Renouvelables Variables (ERV)**

## 🎯 **Objectifs du Projet**

1. **Évaluer la production horaire des installations photovoltaïques** dans les régions de la France métropolitaine en utilisant des données climatiques et des observations des facteurs de capacité.

2. **Prédire la production énergétique des ERV à l'avance**, afin de mieux anticiper les variations de puissance et optimiser la gestion du réseau électrique.




---

In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Liste des URLs pour chaque région
urls = {
    "Auvergne-Rhône-Alpes": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Auvergne-Rh%C3%B4ne-Alpes%22",
    "Bourgogne-Franche-Comté": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Bourgogne-Franche-Comt%C3%A9%22",
    "Bretagne": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Bretagne%22",
    "Centre-Val de Loire": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Centre-Val%20de%20Loire%22",
    "Corse": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Corse%22",
    "Grand Est": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Grand%20Est%22",
    "Hauts-de-France": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Hauts-de-France%22",
    "Île-de-France": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22%C3%8Ele-de-France%22",
    "Normandie": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Normandie%22",
    "Nouvelle-Aquitaine": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Nouvelle-Aquitaine%22",
    "Occitanie": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Occitanie%22",
    "Pays de la Loire": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Pays%20de%20la%20Loire%22",
    "Provence-Alpes-Côte d'Azur": "/api/explore/v2.1/catalog/datasets/production-regionale-mensuelle-filiere/records?select=mois%2Cregion%2Cproduction_solaire&where=mois%3C2022&limit=100&refine=region%3A%22Provence-Alpes-C%C3%B4te%20d'Azur%22"
}

# Base URL pour l'API
base_url = "https://odre.opendatasoft.com"

# Configurer le retry pour gérer les échecs temporaires
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# Liste pour collecter les données de toutes les régions
all_data = []

# Plage de dates filtrées
date_range_start = "2014-01"
date_range_end = "2021-12"

for region, endpoint in urls.items():
    try:
        # Effectuer une requête GET pour chaque région
        response = session.get(base_url + endpoint, timeout=30)
        response.raise_for_status()  # Vérifier si la requête a réussi

        # Charger les données JSON
        data = response.json()
        results = data.get("results", [])

        # Extraire les champs pertinents en filtrant par date
        formatted_data = [
            {
                "mois": record.get("mois"),
                "region": region,
                "production_solaire": record.get("production_solaire")
            }
            for record in results
            if date_range_start <= record.get("mois", "") <= date_range_end
        ]

        # Ajouter les données de cette région à la liste globale
        all_data.extend(formatted_data)

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des données pour {region} : {e}")

# Créer un DataFrame pandas avec toutes les données
df = pd.DataFrame(all_data)

# Réorganiser le format du tableau pour avoir les régions comme colonnes
df_pivot = df.pivot(index="mois", columns="region", values="production_solaire").reset_index()
df_pivot.columns.name = None  # Supprimer le nom des colonnes multi-index

# Exporter les données au format CSV
output_path = "production_solaire_2014_2022.csv"
df_pivot.to_csv(output_path, index=False)

output_path  # Chemin du fichier exporté


'production_solaire_2014_2022.csv'